# Tool Calling Use Cases

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Get API keys

In [2]:
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## Load helper functions

In [4]:
from utils import llama31
from utils import cprint
import json

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Define tool system prompt

In [8]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

09 December 2024


In [9]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

## The brave_search built-in tool

In [10]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [12]:
response = llama31(prompt)
print(response)



brave_search.call(query="current weather in Menlo Park, California")


In [13]:
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [ ]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)

### Calling the search API

In [14]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1733727802, 'localtime': '2024-12-08 23:03'}, 'current': {'last_updated_epoch': 1733727600, 'last_updated': '2024-12-08 23:00', 'temp_c': 9.1, 'temp_f': 48.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 56, 'wind_dir': 'ENE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 0, 'feelslike_c': 9.3, 'feelslike_f': 48.7, 'windchill_c': 9.8, 'windc

### Reprompting Llama with search tool response

In [15]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1733727802, 'localtime': '2024-12-08 23:03'}, 'current': {'last_updated_epoch': 1733727600, 'last_updated': '2024-12-08 23:00', 'temp_c': 9.1, 'temp_f': 48.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 56, 'wind_dir': 'ENE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 0, 'feelslike_c': 9.3, 'feelslike_f': 48.7, 'windchill_c': 9.8, 'windchill_f': 49.7, 'heatindex_c': 9.6, 'heatindex_f': 49.4, 'dewpoint_c': 6.8, 'dewpoint_f': 44.2, 'vis_km': 14.0, 'vis_miles': 8.0, 'uv': 0.0, 'gust_mph': 2.3, 'gust_kph': 3.7}}


In [16]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 09 December 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>

brave_search.call(query="current weather in Menlo Park, California")<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1733727802, 'localtime': '2024-12-08 23:03'}, 'current': {'last_updated_epoch': 1733727600, 'last_updated': '2024-12-08 23:00', 'temp_c': 9.1, 'temp_f': 48.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 5

In [ ]:
response = llama31(prompt)
print(response)

## Using the higher-level message

In [17]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [21]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Lusaka, Zambia?"}
  ]

In [22]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Lusaka, Zambia")


In [ ]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Menlo Park, California?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result}
  ]

In [20]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Menlo Park, California")


## The Wolfram Alpha tool

In [23]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0?"

In [24]:
messages = [
    {"role": "system",  "content": system_prompt_content},
    {"role": "user",    "content": math_problem}
]

In [25]:
response = llama31(messages)
print(response)

import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the equation
equation = x**3 - 2*x**2 - x + 2

# Solve the equation
solutions = sp.solve(equation, x)

print(solutions)


### Calling the Wolfram Alpha tool

In [26]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


### Checking the result

In [27]:
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


### Reprompting Llama with Wolfram Alpha tool response

In [28]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [29]:
response = llama31(messages)
print(response)

The equation x^3 - 2x^2 - x + 2 = 0 has three solutions: x = 1, x = -1, and x = 2.


## The code interpreter built-in tool

In [30]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [31]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]

In [32]:
response = llama31(messages)
print(response)

def calculate_mortgage_payment(principal, annual_interest_rate, years):
    monthly_interest_rate = annual_interest_rate / 1200
    number_of_payments = years * 12
    monthly_payment = principal * monthly_interest_rate * (1 + monthly_interest_rate) ** number_of_payments / ((1 + monthly_interest_rate) ** number_of_payments - 1)
    return monthly_payment

def calculate_total_payment(monthly_payment, years):
    total_payment = monthly_payment * years * 12
    return total_payment

def calculate_total_interest(total_payment, down_payment):
    total_interest = total_payment - down_payment
    return total_interest

# Given values
principal = 1000000  # $1M
annual_interest_rate = 6  # 6%
years = 30
down_payment = 0.2 * principal  # 20% down payment

# Calculate monthly payment
monthly_payment = calculate_mortgage_payment(principal - down_payment, annual_interest_rate, years)

# Calculate total payment
total_payment = calculate_total_payment(monthly_payment, years)

# Calculate total inte

# Calculate loan payments

In [33]:
from utils import calculate_loan
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000, 
    annual_interest_rate = 0.06, 
    loan_term = 30, 
    down_payment = 200000)

print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


## Generating the code in Java

In [34]:
messages = [
    {"role":    "system",     
     "content": system_prompt_content + "\nGenerate the code in Java."},
    {"role":    "user",       "content": loan_question},
  ]

In [36]:
response = llama31(messages)
print(response)

import java.util.Scanner;

public class MortgageCalculator {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);

        // Input values
        double principal = 1000000; // $1M
        double downPayment = 0.2 * principal; // 20% of $1M
        double loanAmount = principal - downPayment; // $800,000
        double annualInterestRate = 0.06; // 6%
        int years = 30;

        // Calculate monthly interest rate
        double monthlyInterestRate = annualInterestRate / 12;

        // Calculate number of payments
        int numberOfPayments = years * 12;

        // Calculate monthly payment
        double monthlyPayment = loanAmount * (monthlyInterestRate * Math.pow(1 + monthlyInterestRate, numberOfPayments)) / (Math.pow(1 + monthlyInterestRate, numberOfPayments) - 1);

        // Calculate total payment
        double totalPayment = monthlyPayment * numberOfPayments;

        // Calculate total interest paid
        double totalIntere

### Reprompting Llama with Code Interpreter tool response

In [37]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [38]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}
  ]

In [39]:
response = llama31(messages)
print(response)

The monthly payment for a 30-year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705. The total interest paid over the life of the loan is $926,705.


## Llama 3.1 custom tool calling

In [43]:
from utils import trending_songs, get_boiling_point

country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>trending_songs()</code> in <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Prompt Llama 3.1 for custom tool call

In [46]:
user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US?
"""

In [47]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt}
  ]

In [48]:
result = llama31(messages,405)
print(result)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool

In [ ]:
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [51]:
res = json.loads(result)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [50]:
tool_result = custom_tools[function_name](*parameters)
tool_result

NameError: name 'custom_tools' is not defined

### Reprompting Llama with custom tool call result

In [52]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": result},
    {"role": "ipython", "content": ','.join(tool_result)}
  ]

In [53]:
response = llama31(messages, 70)
print(response)

The function call returned two values, but the values themselves are not provided in the output. However, based on the function description, the output should be the top 5 trending songs in the US.
